# Parameter exploration

In [1]:
import denest
import nest
import yaml
from pathlib import Path
from pprint import pprint

In [2]:
from denest import ParamsTree

In [3]:
# Path to the parameter files to use
PARAMS_PATH = './data/params/tree_paths.yml'

## Override parameters

#### Load  "base" parameter files

In [4]:
base_tree = denest.load_trees(PARAMS_PATH)

2020-06-30 13:44:15,301 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:44:15,306 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:44:15,311 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']


#### Define some overrides from dictionary

In [5]:
# We can override some parameters loaded from the file
overrides = [

  # Maybe change the nest kernel's settings ?
  {'kernel': {'nest_params': {'local_num_threads': 20}}},

  # Maybe change a parameter for all the projections at once ?
  {'network': {'projection_models': {'nest_params': {
      'allow_autapses': True
  }}}},

]

#### Define some overrides using the ``AutoDict`` class

The ``AutoDict`` class provides a convenient way of creating deeply nested dictionaries

In [6]:
from denest.utils.autodict import AutoDict

In [7]:
override = AutoDict({
    ('kernel', 'nest_params', 'local_num_threads'): 20,
    ('network', 'projection_models', 'nest_params', 'allow_autapses'): 20,
}).todict()

overrides = [override]

#### Load overriden parameter tree

The `denest.load_trees` method supports overrides:

In [8]:
overriden_tree = denest.load_trees(PARAMS_PATH, *overrides)

2020-06-30 13:44:20,236 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:44:20,237 [denest] INFO: Using 1 override tree(s)
2020-06-30 13:44:20,241 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:44:20,245 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']


The data defined in the overrides were inserted at the corresponding nodes:

In [9]:
print('Base params:')
print(f"kernel nest_params: ``{base_tree.children['kernel'].nest_params}``")
print(f"root 'projection_models' nest_params: ``{base_tree.children['network'].children['projection_models'].nest_params}``")
print('...')
print('Overriden params:')
print(f"kernel nest_params: ``{overriden_tree.children['kernel'].nest_params}``")
print(f"root 'projection_models' nest_params: ``{overriden_tree.children['network'].children['projection_models'].nest_params}``")

Base params:
kernel nest_params: ``DeepChainMap({'resolution': 0.5, 'overwrite_files': True}, {})``
root 'projection_models' nest_params: ``DeepChainMap({'connection_type': 'divergent', 'mask': {'circular': {'radius': 2.0}}, 'kernel': 1.0}, {}, {})``
...
Overriden params:
kernel nest_params: ``DeepChainMap({'resolution': 0.5, 'overwrite_files': True, 'local_num_threads': 20}, {})``
root 'projection_models' nest_params: ``DeepChainMap({'connection_type': 'divergent', 'mask': {'circular': {'radius': 2.0}}, 'kernel': 1.0, 'allow_autapses': 20}, {}, {})``


#### Or merge parameter trees manually using the ``ParamsTree.merge`` method

In [10]:
overriden_tree_2 = ParamsTree.merge(
    base_tree,
    *[
        ParamsTree(override)
        for override in overrides
    ]
)

In [11]:
print(overriden_tree_2 == overriden_tree)

True


## Perform parameter exploration

We can run multiple simulations using overrides:

In [12]:
param_names = [
    'g_peak_AMPA_all_neurons',
    'weights_proj_1_AMPA',
    'weights_proj_2_GABAA',
]

# Values of the parameters
explored_values = (
    [0.1, 0.2], # 'g_peak_AMPA_all_neurons',
    [1.0, 2.0], # 'weights_proj_1_AMPA',
    [1.0, 2.0], # 'weights_proj_2_GABA_A',
)

# Position in the tree of the modified params
# Be careful not to make any mistake here!!
params_paths = {
    'g_peak_AMPA_all_neurons': ('network', 'neuron_models', 'my_neuron', 'nest_params', 'g_peak_AMPA'),
    'weights_proj_1_AMPA': ('network', 'projection_models', 'proj_1_AMPA', 'nest_params', 'weights'),
    'weights_proj_2_GABAA': ('network', 'projection_models', 'proj_2_GABAA', 'nest_params', 'weights'),
}

In [13]:
import itertools

# We save all the simulations there
MAIN_OUTPUT_DIR = Path('./data/outputs/output_param_explore')

# Run one simulation for each  combination of parameters
for param_values_combination in itertools.product(*explored_values):
    
    param_combination = dict(zip(param_names, param_values_combination))
    print(f'param combination: {param_combination}', flush=True)
    
    # Create override tree to incorporate the parameters
    overrides = [

        # Incorporate parameter exploration params
        AutoDict({
            params_paths[key]: value
            for key, value in param_combination.items()
        }).todict(),  
        
        # Reduce the multimeter interval for disk space
        AutoDict({
            ('network', 'recorder_models', 'my_multimeter', 'nest_params', 'interval'): 50.0
        }).todict(),  

    ]
    print(f'Override trees:\n {overrides}', flush=True)
    
    # Load overriden parameter file
    overriden_tree = denest.load_trees(PARAMS_PATH, *overrides)
    
    # Define output directory for current simulation
    output_dir = MAIN_OUTPUT_DIR/str(param_combination)
    print(output_dir)
    
    # Run simulation
    print(f'Running simulation for param combination: {param_combination}')
    sim = denest.Simulation(overriden_tree, output_dir=output_dir)
    sim.run()
    print(f'done\n\n')

param combination: {'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
Override trees:
 [{'network': {'neuron_models': {'my_neuron': {'nest_params': {'g_peak_AMPA': 0.1}}}, 'projection_models': {'proj_1_AMPA': {'nest_params': {'weights': 1.0}}, 'proj_2_GABAA': {'nest_params': {'weights': 1.0}}}}}, {'network': {'recorder_models': {'my_multimeter': {'nest_params': {'interval': 50.0}}}}}]


2020-06-30 13:44:23,300 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:44:23,301 [denest] INFO: Using 2 override tree(s)
2020-06-30 13:44:23,307 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:44:23,328 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-30 13:44:23,452 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-30 13:44:23,453 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-30 13:44:23,454 [denest.simulation] INFO:   Resetting NEST kernel...


data/outputs/output_param_explore/{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
Running simulation for param combination: {'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}


2020-06-30 13:44:23,487 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-30 13:44:23,538 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-30 13:44:23,575 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': "data/outputs/output_param_explore/{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}/data", 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-06-30 13:44:23,645 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-06-30 13:44:23,651 [denest.simulation] INFO:   Installing external modules...
2020-06-30 13:44:23,651 [denest.simulation] INFO:   Finished installing external modules
2020-06-30 13:44:23,654 [denest.simulation] INFO: Finished initializing kernel
2020-06-30 13:44:23,678 [denest.simulation] INFO: Build N=3 session models
2020-06-30 13:44:23,683 [denest.simulation] INFO: Build N=4 sessions
2020-06-30 13:44:23,685 [den

2020-06-30 13:44:25,207 [denest.session] INFO: Finished running session
2020-06-30 13:44:25,215 [denest.session] INFO: Session '01_3_spikes' virtual running time: 100 ms
2020-06-30 13:44:25,224 [denest.session] INFO: Session '01_3_spikes' real running time: 0h:00m:00s
2020-06-30 13:44:25,246 [denest.simulation] INFO: Done running session '01_3_spikes'
2020-06-30 13:44:25,250 [denest.simulation] INFO: Running session: '02_2_spikes'...
2020-06-30 13:44:25,277 [denest.session] INFO: Initializing session...
2020-06-30 13:44:25,282 [denest.session] INFO: Setting `origin` flag to `200.0` for all stimulation devices in ``InputLayers`` for session `02_2_spikes`
2020-06-30 13:44:25,296 [denest.utils.validation] INFO: Object `Unit changes dictionary`: params: using default value for optional parameters:
{'change_type': 'constant', 'from_array': False}
2020-06-30 13:44:25,298 [denest.network.layers] INFO: Layer='input_layer', pop='spike_generator': Applying 'constant' change, param='spike_times',

...

done


param combination: {'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
Override trees:
 [{'network': {'neuron_models': {'my_neuron': {'nest_params': {'g_peak_AMPA': 0.2}}}, 'projection_models': {'proj_1_AMPA': {'nest_params': {'weights': 2.0}}, 'proj_2_GABAA': {'nest_params': {'weights': 2.0}}}}}, {'network': {'recorder_models': {'my_multimeter': {'nest_params': {'interval': 50.0}}}}}]


2020-06-30 13:44:38,835 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:44:38,843 [denest] INFO: Using 2 override tree(s)
2020-06-30 13:44:38,846 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:44:38,847 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-30 13:44:38,903 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-30 13:44:38,904 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-30 13:44:38,904 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-30 13:44:38,914 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-30 13:44:38,916 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-30 13:44:38,919 [denest.simulation] INFO:     Calling `nest.SetKerne

data/outputs/output_param_explore/{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
Running simulation for param combination: {'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}


2020-06-30 13:44:39,026 [denest.network] INFO: Creating neuron models...
100%|██████████| 2/2 [00:00<00:00, 2840.71it/s]
2020-06-30 13:44:39,032 [denest.network] INFO: Creating synapse models...
100%|██████████| 2/2 [00:00<00:00, 1960.41it/s]
2020-06-30 13:44:39,043 [denest.network] INFO: Creating recorder models...
100%|██████████| 3/3 [00:00<00:00, 2373.24it/s]
2020-06-30 13:44:39,049 [denest.network] INFO: Creating layers...
100%|██████████| 2/2 [00:00<00:00,  7.59it/s]
2020-06-30 13:44:39,327 [denest.network] INFO: Creating population recorders...
100%|██████████| 2/2 [00:00<00:00, 86.03it/s]
2020-06-30 13:44:39,354 [denest.network] INFO: Creating projection recorders...
100%|██████████| 1/1 [00:00<00:00, 423.54it/s]
2020-06-30 13:44:39,360 [denest.network] INFO: Connecting layers...
100%|██████████| 3/3 [00:00<00:00, 342.90it/s]
2020-06-30 13:44:39,372 [denest.network] INFO: Network size (including recorders and parrot neurons):
Number of nodes: 206
Number of projections: 6650
202

done




In [14]:
!ls {MAIN_OUTPUT_DIR}

{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 2.0}
{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 2.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
